In [1]:
%pip install langchain pypdf openai tiktoken pinecone langchain-nvidia-ai-endpoints -U -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\anubh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
%pip install -U langchain-community -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\anubh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# %pip install pinecone-client langchain-pinecone -U
# %pip uninstall pinecone-client pinecone
# %pip install pinecone

In [3]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
# from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os


#Loading the words/sentences

In [4]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()
data[0]

Document(metadata={'producer': 'iText 2.0.8 (by lowagie.com)', 'creator': 'PyPDF', 'creationdate': '2024-11-02T22:24:12+00:00', 'moddate': '2024-11-02T22:24:12+00:00', 'source': 'pdfs\\invoice (1).pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Tax Invoice/Bill of Supply/Cash Memo\n(Original for Recipient)\n*ASSPL-Amazon Seller Services Pvt. Ltd., ARIPL-Amazon Retail India Pvt. Ltd. (only where Amazon Retail India Pvt. Ltd. fulfillment center is co-located) \nCustomers desirous of availing input GST credit are requested to create a Business account and purchase on Amazon.in/business from Business eligible offers \nPlease note that this invoice is not a demand for payment\nPage 1 of 1\nFor CLICKTECH RETAIL PRIVATE LIMITED:\nAuthorized Signatory\nOrder Number:404-2866309-6187566 Invoice Number :FCCB-112980\nOrder Date:03.11.2024 Invoice Details :WB-FCCB-297683823-2425\nInvoice Date :03.11.2024\nSl.\nNo Description Unit\nPrice Qty Net\nAmount\nTax\nRate\nTax\nType\nTa

#Tokenzation

In [5]:
"""
This object is used to split a large text into smaller chunks(tokenization).
1. chunk_size=500: This parameter sets the maximum number of characters in each chunk to 500.
2. chunk_overlap=0: This parameter sets the number of characters that overlap between consecutive chunks to 0. This means the chunks will not have any overlapping text.
"""
text_spiltter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
text_chunks = text_spiltter.split_documents(data)
# Access the page_content of each document in the list
for i, chunk in enumerate(text_chunks):
    print(f"Chunk {i}: {chunk.page_content[:200]}...") # Print first 200 characters of each chunk

# text_chunks[0].page_content

Chunk 0: Tax Invoice/Bill of Supply/Cash Memo
(Original for Recipient)
*ASSPL-Amazon Seller Services Pvt. Ltd., ARIPL-Amazon Retail India Pvt. Ltd. (only where Amazon Retail India Pvt. Ltd. fulfillment center ...
Chunk 1: Authorized Signatory
Order Number:404-2866309-6187566 Invoice Number :FCCB-112980
Order Date:03.11.2024 Invoice Details :WB-FCCB-297683823-2425
Invoice Date :03.11.2024
Sl.
No Description Unit
Price Q...
Chunk 2: 9% SGST ₹60.94
TOTAL: ₹121.88₹799.00
Amount in Words:
Seven Hundred Ninety-nine only
Whether tax is payable under reverse charge - No
Sold By :
CLICKTECH RETAIL PRIVATE LIMITED 
*Govindapur, Benepukur...
Chunk 3: Shipping Address :
Pushpita Biswas 
Ankit Kumar Sinha 
1st floor, flat D, Annapurna Bhawan,AG-188,
AG-188,Hana para road, Hanapara, Keshtopur 
KOLKATA, WEST BENGAL, 700102 
IN 
State/UT Code:19 
Place...


In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_KEY = os.getenv("PINECONE_API_KEY")
NVIDIA_KEY = os.getenv("NVIDIA_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_API_ENV")
NVIDIA_KEY = os.getenv("NVIDIA_API_KEY")

We will use nvidia api key becoz thats free and openai is not`

In [8]:

embeddings = NVIDIAEmbeddings(
  model="NV-Embed-QA", 
  api_key=NVIDIA_KEY, 
  truncate="NONE", 
  )

embed = embeddings.embed_query("Ayo , how are you doing?")
len(embed)


1024

Getting the vectors from this api and store them in the pinecone DB

In [9]:
index_name = "anubhav"

In [10]:
#initializing pinecone
from pinecone import Pinecone
# pc = Pinecone(
#     api_key=PINECONE_KEY
# )

pc = Pinecone(api_key=PINECONE_KEY)
index = pc.Index(index_name)

Exception: The official Pinecone python package has been renamed from `pinecone-client` to `pinecone`. Please remove `pinecone-client` from your project dependencies and add `pinecone` instead. See the README at https://github.com/pinecone-io/pinecone-python-client for more information on using the python SDK.

Create the embeddings for each of the text chunks

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_texts(
    [chunk.page_content for chunk in text_chunks],
    embeddings,
    index_name=index_name
)
vector_store

In [ ]:
#similarity search
query = "Pushpita Biswas is my sister"
docs = vector_store.similarity_search(query)

In [ ]:
print(docs[0].page_content)

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

client = ChatNVIDIA(
  model="deepseek-ai/deepseek-r1",
  api_key=NVIDIA_KEY, 
  temperature=0.6,
  top_p=0.7,
  max_tokens=4096,
)

response = client.invoke([{"role":"user","content":"Hello"}])
print(response.content)